# Deep Learning: Computer Vision Using Transfer Learning Method
> AHSNCCU/NTNU CSIE 王修佑

## 個人化狗門

以`presidential_doggy_door`資料夾中"第一狗狗Bo"為目標建立模型

<img src="presidential_doggy_door/train/bo/bo_10.jpg">

+ 現況：  
Dataset小>>>model會Overfitting或Generalization能力差

+ Solution:  
Transfer Learning

## 下載預先訓練模型

+ 運用[ImageNet 預先訓練模型](https://keras.io/api/applications/vgg/#vgg16-function)
+ Task: 改變最後一個Dense Layer，設定旗標 `include_top=False`

In [1]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False)

In [2]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

## 凍結基本模型

In [3]:
base_model.trainable = False

## 新增模型層

+ 新增新的可訓練層
    + Pooling Layer
    + Dense Layer

In [4]:
inputs = keras.Input(shape=(224, 224, 3))
# Separately from setting trainable on the model, we set training to False 
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

查看新的模型

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 513       
                                                                 
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________


## 編寫模型

+ [二元交叉熵](https://www.tensorflow.org/api_docs/python/tf/keras/losses/BinaryCrossentropy)

+ `from_logits=True`: [損失函數](https://gombru.github.io/2018/05/23/cross_entropy_loss/)輸出值不需要被正規化 (例如透過 softmax)

In [ ]:
# Important to use binary crossentropy and binary accuracy as we now have a binary classification problem
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True), metrics=[keras.metrics.BinaryAccuracy()])

## 增強資料

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# create a data generator
datagen = ImageDataGenerator(
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False) # we don't expect Bo to be upside-down so we will not flip vertically

## 載入資料

使用 Keras 的 [`flow_from_directory`](https://keras.io/api/preprocessing/image/)載入影像

[flow_from_directory](https://keras.io/api/preprocessing/image/) 可以調整影像大小來符合模型：244x244 像素和 3 個色頻。

In [ ]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('presidential_doggy_door/train/', 
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode='binary', 
                                       batch_size=8)
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('presidential_doggy_door/valid/', 
                                      target_size=(224, 224), 
                                      color_mode='rgb', 
                                      class_mode='binary', 
                                      batch_size=8)

## 訓練模型

設定 `steps_per_epoch`的數量：

In [ ]:
model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, epochs=20)

---

## 微調模型

現在模型的新層經過訓練後，我們可以選擇應用最後一項技巧來改善模型，也就是[微調](https://developers.google.com/machine-learning/glossary#f)。微調時我們要先取消凍結整個模型，並採用非常小的[學習率](https://developers.google.com/machine-learning/glossary#learning-rate)再次訓練模型。這樣一來，預先訓練的基本層會有非常微量的進步並經過稍微調整，以小幅度的方式改善模型。

請注意，有凍結層的模型一定要經過完整訓練後，才執行此步驟。我們先前新增至模型的未經訓練池化層和分類層，都是經過隨機初始化。這表示這兩層需要大幅更新才能正確分類影像。在整個[反向傳播](https://developers.google.com/machine-learning/glossary#backpropagation)過程中，最後一層的大型初始更新會導致預先訓練層也需要大幅更新。這些更新可能會破壞預先訓練的重要功能。然而，在最後兩層經過訓練且聚合的情況下，對整體模型的任何更新都會小得多 (尤其是非常小的學習率)，而不會破壞前幾層的功能。

讓我們試著取消凍結預先訓練的模型層，然後微調模型：

In [ ]:
# Unfreeze the base model
base_model.trainable = True

# It's important to recompile your model after you make any changes
# to the `trainable` attribute of any inner layer, so that your changes
# are taken into account
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),  # Very low learning rate
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, epochs=10)

## 檢查預測結果

現在我們的模型已經過完整訓練，可以為 Bo 建立狗門了！首先我們來看看模型的預測。我們要用和上次建立狗門相同的方式來預先處理影像。

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing import image as image_utils
from tensorflow.keras.applications.imagenet_utils import preprocess_input

def show_image(image_path):
    image = mpimg.imread(image_path)
    plt.imshow(image)

def make_predictions(image_path):
    show_image(image_path)
    image = image_utils.load_img(image_path, target_size=(224, 224))
    image = image_utils.img_to_array(image)
    image = image.reshape(1,224,224,3)
    image = preprocess_input(image)
    preds = model.predict(image)
    return preds

用幾張影像試試看，然後看看預測結果如何：

In [ ]:
make_predictions('presidential_doggy_door/valid/bo/bo_20.jpg')

In [ ]:
make_predictions('presidential_doggy_door/valid/not_bo/121.jpg')

看來負數預測結果表示是 Bo，而正數預測結果則表示是其他寵物。我們可以用這項資訊來控制狗門只讓 Bo 進來！

## 練習：Bo 的狗門

請填入下列程式碼以實作 Bo 的狗門：

In [ ]:
def presidential_doggy_door(image_path):
    preds = make_predictions(image_path)
    if FIXME:
        print("It's Bo! Let him in!")
    else:
        print("That's not Bo! Stay out!")

## 解決方案

按一下以下的「...」來查看解決方案。

```python
def presidential_doggy_door(image_path):
    preds = make_predictions(image_path)
    if preds[0] < 0:
        print("It's Bo! Let him in!")
    else:
        print("That's not Bo! Stay out!")
```

來試試看吧！

In [ ]:
presidential_doggy_door('presidential_doggy_door/valid/not_bo/131.jpg')

In [ ]:
presidential_doggy_door('presidential_doggy_door/valid/bo/bo_29.jpg')

## 摘要

做得好！透過遷移學習，你使用非常小的資料集就打造出高度準確的模型。這可以是極為強大的技術，也能決定一項專案是成功還是無法實現。我們希望這些技術能在未來的類似案例中幫助到你！

在 [NVIDIA 遷移學習工具組](https://developer.nvidia.com/tlt-getting-started)中有豐富的遷移學習相關實用資源。

### 清除記憶體
在繼續之前，請執行下列儲存格以清除 GPU 記憶體。

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## 下一步

到目前為止，此實作坊的重點主要是影像分類。在下一節中，為了更全面地介紹深度學習，我們要切換主題並討論與序列資料相關的作業，這需要用到不同的方法。

請繼續前往下一節：[*序列資料*](./06_headline_generator.ipynb)。